# Build Pivot Tables in Excel Using Python

__Summary:__ In this tutorial, we shall use Python to build pivot tables in excel. This could be a really handy tool for report automation in Analytics domian

In [1]:
import win32com.client as win32
from pywintypes import com_error
from pathlib import Path
import sys
import os
# Required to generate random data for simulation
import random
import pandas as pd
import numpy as np
from datetime import datetime, date

In [2]:
os.chdir("../Files")

In [3]:
# Get win32 constants
win32c = win32.constants

### Steps:
[1 Generate Data](#GD)<br>
[2 Function to create Pivot Table](#FPT)<br>
[3 Create excel file and call Pivot table function](#CPTF)<br>
[4 Main function](#MF)<br>

<a id="GD"></a>
### 1. Generate Data<br>

As a first step, we shall generate random data in excel file. If you already have a excel file with data, feel free to skip this part. We shall generate a data of size 10,000 with below four columns
<ol>
    <li>Date: Random date</li>
    <li>Expense: Type - String; Options - "Personal", "Business", or "Misc"</li>
    <li>Products: Type - String; Options - "Coffee", "Dinner", "Stationary", "Fuel"</li>
    <li>Price: Type - int, normally distributed with mean: 25, and standard deviation: 10</li>
    </ol>

In [4]:
def create_excel_file(f_path: Path, f_name: str, sheet_name: str):
    """
    f_path: Path where to store the excel file
    f_name: Name of the excel file
    sheet_name: Excel sheet name (where synthetic data will be stored)
    """
    filename = f_path / f_name
    random.seed(365)
    np.random.seed(365)
    number_of_data_rows = 10000
    
    # create list of 31 dates
    dates = pd.bdate_range(date.today(), freq='1d', periods=31).tolist()

    # Create random data points
    data = {'Date': [random.choice(dates) for _ in range(number_of_data_rows)],
            'Expense': [random.choice(['Personal', 'Business', 'Misc']) for _ in range(number_of_data_rows)],
            'Products': [random.choice(['Coffee', 'Dinner', 'Stationary', 'Fuel']) for _ in range(number_of_data_rows)],
            'Price': np.random.normal(25, 10, size=(1, number_of_data_rows))[0]}

    # create the dataframe and save it to Excel
    pd.DataFrame(data).to_excel(filename, index=False, sheet_name=sheet_name, float_format='%.4f')
    return

<a id="FPT"></a>
### 2. Function to create Pivot Table<br>

__Summary:__ Below function creates a pivot table in excel given the defined inputs

In [5]:
def pivot_table(wb: object, ws: object, pt_ws: object, ws_name: str, pt_name: str,
                pt_rows: list, pt_cols: list, pt_filters: list, pt_fields: list):
    """
    wb: workbook reference
    ws: worksheet with data (to be used in pivot table)
    pt_ws: pivot table worksheet number
    ws_name: pivot table worksheet name
    pt_name: pivot table name
    pt_rows: rows of pivot table
    pt_cols: columns of pivot table
    pt_filters: filters of pivot table
    pt_fields: values selected for filling the pivot tables
    """

    # pivot table location
    pt_loc = len(pt_filters) + 2
    
    # grab the pivot table source data
    pc = wb.PivotCaches().Create(SourceType=win32c.xlDatabase, SourceData=ws.UsedRange)
    
    # create the pivot table object
    pc.CreatePivotTable(TableDestination=f'{ws_name}!R{pt_loc}C1', TableName=pt_name)

    # selecte the pivot table work sheet and location to create the pivot table
    pt_ws.Select()
    pt_ws.Cells(pt_loc, 1).Select()

    # Sets the rows, columns and filters of the pivot table
    for field_list, field_r in ((pt_filters, win32c.xlPageField), (pt_rows, win32c.xlRowField),
                                (pt_cols, win32c.xlColumnField)):
        for i, value in enumerate(field_list):
            pt_ws.PivotTables(pt_name).PivotFields(value).Orientation = field_r
            pt_ws.PivotTables(pt_name).PivotFields(value).Position = i + 1

    # Sets the Values of the pivot table
    for field in pt_fields:
        pt_ws.PivotTables(pt_name).AddDataField(pt_ws.PivotTables(pt_name).PivotFields(field[0]),
                                                field[1], field[2]).NumberFormat = field[3]

    # Visiblity True or Valse
    pt_ws.PivotTables(pt_name).ShowValuesRow = True
    pt_ws.PivotTables(pt_name).ColumnGrand = True
    return

<a id="CPTF"></a>
### 3. Create excel file and call Pivot table function<br>

__Summary:__ The function does below 4 tasks:
<ol>
    <li>Opens the excel file</li>
    <li>Creates a pivot table worksheet</li>
    <li>Calls the pivot function</li>
    <li>Close the excel workbook, and save the results</li>
    </ol>

In [6]:
def run_excel(f_path: Path, f_name: str, sheet_name: str):

    filename = f_path / f_name

    # create excel object
    excel = win32.gencache.EnsureDispatch('Excel.Application')

    # excel can be visible or not
    excel.Visible = False  # False
    
    # try except for file / path
    try:
        wb = excel.Workbooks.Open(filename)
    except com_error as e:
        if e.excepinfo[5] == -2146827284:
            print(f'Failed to open spreadsheet.  Invalid filename or location: {filename}')
        else:
            raise e
        sys.exit(1)

    # set worksheet
    ws1 = wb.Sheets('data')
    
    # Setup and call pivot_table
    ws2_name = 'pivot_table'
    wb.Sheets.Add().Name = ws2_name
    ws2 = wb.Sheets(ws2_name)
    
    pt_name = 'Example'  # must be a string
    pt_rows = ['Expense']  # must be a list
    pt_cols = ['Products']  # must be a list
    pt_filters = ['Date']  # must be a list
    # [0]: field name [1]: pivot table column name [3]: calulation method [4]: number format
    pt_fields = [['Price', 'Price: mean', win32c.xlAverage, '$#,##0.00'],  # must be a list of lists
                 ['Price', 'Price: sum', win32c.xlSum, '$#,##0.00'],
                 ['Price', 'Price: count', win32c.xlCount, '0']]
    
    pivot_table(wb, ws1, ws2, ws2_name, pt_name, pt_rows, pt_cols, pt_filters, pt_fields)
    
    # Close the excel file and save the changes
    wb.Close(SaveChanges=1)
    return

<a id="MF"></a>
### 4. Main function<br>

__Summary:__ This function does below tasks:
<ol>
    <li>Create a Excel file with given name with data (calling function: create_test_excel_file)</li>
    <li>Create pivot table (by calling: run_excel function)</li>
    </ol>

In [7]:
def main():
    # sheet name for data
    sheet_name = 'data'  # update with sheet name from your file
    # file path
    f_path = Path.cwd()  # file in current working directory
#   f_path = Path(r'c:\...\Documents')  # file located somewhere else
    # excel file
    f_name = 'Test.xlsx'
    
    # function calls
    create_excel_file(f_path, f_name, sheet_name)  # remove when running your own file
    run_excel(f_path, f_name, sheet_name)

In [8]:
main()